#### 데이터 전처리 및 시각화

##### 전처리
데이터프레임 병합

In [59]:
# import
import pandas as pd
import numpy as np

import folium
import json

In [60]:
# 스타벅스 매장리스트
sbuck_df = pd.read_excel('./seoul_starbuks.xlsx', header=0)
sbuck_df.head()

,매장명,위도,경도,매장타입,주소
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동)
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동)
2,신사역성일빌딩,37.513931,127.020606,general,서울특별시 강남구 강남대로 584 (논현동)
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동)
4,대치재경빌딩R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동)


In [61]:
sbuck_df['주소'][599].split()[1]

'중랑구'

In [62]:
gu_list = []
for item in sbuck_df['주소']:
    gu = item.split()[1]
    gu_list.append(gu)

sbuck_df['구'] = gu_list
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소,구
595,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,중랑구
596,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),중랑구
597,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구
598,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구
599,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35,중랑구


##### 서울시 행정구역별 위경도 전처리

- 대한민국 행정구역 위경도엑셀파일에서 서울만 필터링

In [63]:
seoul_loc_df = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
seoul_loc_df.head()

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275


In [64]:
seoul_gu_df = seoul_loc_df[seoul_loc_df['do'] == '서울']
seoul_gu_df.head()

,docity,do,city,longitude,latitude
138,서울강남구,서울,강남구,127.049556,37.514575
139,서울강동구,서울,강동구,127.125864,37.527367
140,서울강북구,서울,강북구,127.027719,37.636956
141,서울강서구,서울,강서구,126.851675,37.548156
142,서울관악구,서울,관악구,126.953844,37.475386


In [65]:
seoul_gu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 138 to 162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     25 non-null     object 
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.2+ KB


In [66]:
seoul_gu_df.to_excel('./seoul_gu_info.xlsx', index=False)

##### 서울시 구별 스타벅스 매장수 구하기

In [67]:
sbuck_gu_counts= sbuck_df.pivot_table(index='구',
                                      values='매장명',
                                      aggfunc='count').rename(columns= {'매장명': '스타벅스_매장수'})
sbuck_gu_counts.head()

,스타벅스_매장수
구,
강남구,88
강동구,17
강북구,6
강서구,25
관악구,11


In [68]:
# seoul_gu_df 수정
seoul_gu_df = seoul_gu_df[['do', 'city', 'latitude', 'longitude']]
columns = {'do':'도', 'city':'구', 'latitude':'위도', 'longitude':'경도'}
seoul_gu_df.rename(columns=columns, inplace=True)
seoul_gu_df.reset_index(drop=True, inplace=True)

In [69]:
seoul_gu_df.head()

,도,구,위도,경도
0,서울,강남구,37.514575,127.049556
1,서울,강동구,37.527367,127.125864
2,서울,강북구,37.636956,127.027719
3,서울,강서구,37.548156,126.851675
4,서울,관악구,37.475386,126.953844


In [70]:
seoul_sgg = pd.merge(seoul_gu_df, sbuck_gu_counts, how='left', on='구')
seoul_sgg

,도,구,위도,경도,스타벅스_매장수
0,서울,강남구,37.514575,127.049556,88
1,서울,강동구,37.527367,127.125864,17
2,서울,강북구,37.636956,127.027719,6
3,서울,강서구,37.548156,126.851675,25
4,서울,관악구,37.475386,126.953844,11
5,서울,광진구,37.535739,127.084533,18
6,서울,구로구,37.492650,126.889597,14
7,서울,금천구,37.449108,126.904197,12
8,서울,노원구,37.651461,127.058389,14
9,서울,도봉구,37.665833,127.049522,5


In [71]:
seoul_pop = pd.read_excel('./seoul_pop.xlsx')
seoul_pop.tail()

,구,계
20,관악구,501226
21,서초구,408451
22,강남구,534103
23,송파구,664514
24,강동구,464037


In [72]:
seoul_sgg = pd.merge(seoul_sgg, seoul_pop, how='left', on='구')
seoul_sgg.head()

,도,구,위도,경도,스타벅스_매장수,계
0,서울,강남구,37.514575,127.049556,88,534103
1,서울,강동구,37.527367,127.125864,17,464037
2,서울,강북구,37.636956,127.027719,6,297702
3,서울,강서구,37.548156,126.851675,25,574638
4,서울,관악구,37.475386,126.953844,11,501226


In [73]:
# 사업체수
seoul_biz = pd.read_excel('./seoul_biz.xlsx')
seoul_biz.tail()

,구,사업체수,종사자수계
20,관악구,37915,128417
21,서초구,71958,487976
22,강남구,110007,801419
23,송파구,74531,400781
24,강동구,41660,157465


In [74]:
seoul_sgg_stat = pd.merge(seoul_sgg, seoul_biz, how='left', on='구')
seoul_sgg_stat.tail()

,도,구,위도,경도,스타벅스_매장수,계,사업체수,종사자수계
20,서울,용산구,37.536094,126.967522,25,233284,29680,152605
21,서울,은평구,37.599969,126.931242,13,470602,36509,109031
22,서울,종로구,37.570378,126.981642,40,152211,48361,275063
23,서울,중구,37.561003,126.999642,53,130785,70308,386564
24,서울,중랑구,37.603806,127.094778,8,390140,39310,113144


In [75]:
columns = {'도':'도','구':'구','경도':'경도','위도':'위도','스타벅스_매장수':'스타벅스_매장수',
           '계_x':'인구수','사업체수':'사업체수','종사자수계':'종사자수'}
seoul_sgg_stat.rename(columns=columns, inplace=True)

In [76]:
seoul_sgg_stat.to_excel('./seoul_sgg_stat.xlsx', index=False)

##### 데이터 시각화

In [77]:
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소,구
595,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,중랑구
596,상봉역,37.596890,127.086470,general,서울특별시 중랑구 망우로 307 (상봉동),중랑구
597,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구
598,양원역,37.606654,127.106360,general,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구
599,중화역,37.601709,127.078411,general,서울특별시 중랑구 봉화산로 35,중랑구


In [78]:
sbuck_map = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color='green', fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=3).add_to(sbuck_map)

sbuck_map

In [79]:
sbuck_map2 = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']
    type = sbuck_df.loc[idx, '매장타입']

    # 매장타입별 색상
    fillcolor = ''
    size = 2
    if type == 'general':
        fillcolor = 'gray'
        size = 2
    elif type == 'reserve':
        fillcolor = 'black'
        size = 5
    elif type == 'generalDT':
        fillcolor = 'red'
        size = 3
    elif type == 'generalWT':
        fillcolor = 'blue'
        size = 3

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color=fillcolor, fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=size).add_to(sbuck_map2)

sbuck_map2

In [80]:
# 시군구경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))

sbuck_map3 = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map3)

sbuck_map3

In [81]:
sbuck_mean = seoul_sgg_stat['스타벅스_매장수'].mean()
sbuck_mean

24.0

In [82]:
# 시군구경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))

sbuck_map3 = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map3)

for idx in seoul_sgg_stat.index:
    lat = seoul_sgg_stat.loc[idx, '위도']
    lng = seoul_sgg_stat.loc[idx, '경도']
    count = seoul_sgg_stat.loc[idx, '스타벅스_매장수']

    if count > sbuck_mean: # 구별 평균치보다 매장수가 많음
        fillcolor = 'red'
    else:
        fillcolor = 'blue'

    folium.CircleMarker(location=[lat, lng], color='#ffff00', fill_color=fillcolor, fill_opacity=0.7, weight=1.5, radius=count/2, popup=count).add_to(sbuck_map3)

sbuck_map3

In [83]:
# 시군구경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))

sbuck_map4 = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=10)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map4)

folium.Choropleth(
    geo_data=seoul_sgg_geo,
    data=seoul_sgg_stat,
    columns=['구', '스타벅스_매장수'],
    fill_color='Purples', # YlGn, RdYlGn, Spectral, Accent, Reds, Greens, Blues, Oranges, Purples, Greys, PuOr, OrRd
    fill_opacity=0.7, 
    line_opacity=0.5,
    key_on='properties.name', # 이게 핵심 - key값이 없으면 지도에 색상표시 못함
).add_to(sbuck_map4)

sbuck_map4

In [86]:
# 서울시 인구수별 분포
seoul_pop_choropleth = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=10)

folium.Choropleth(geo_data=seoul_sgg_geo, data=seoul_sgg_stat,
                  columns=['구', '인구수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(seoul_pop_choropleth)

seoul_pop_choropleth

KeyError: '인구수'

- 결론 : 구에 살고 있는 인구수와 스타벅스 매장수는 관계 거의 없다

In [85]:
# 서울시 인구수별 분포
seoul_biz_choropleth = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=10)

folium.Choropleth(geo_data=seoul_sgg_geo, data=seoul_sgg_stat,
                  columns=['구', '사업체수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(seoul_biz_choropleth)

seoul_biz_choropleth

- 결론 : 각 구별 사업체수와 스타벅스 매장수는 깊은 연관이 있다

         동작구는 사당역 근처 리저브 매장을 폐점했다(매출이 부진 예상)